In [ ]:
pip install pydicom nibabel numpy torch torchvision segmentation-models-pytorch scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install albumentations

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from scipy.ndimage import zoom
from operator import itemgetter
from sklearn.metrics import jaccard_score

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

num_classes = 7
batch_size = 8
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"
test_augmentations = A.Compose([
    A.Resize(height=256, width=256),
    A.Normalize(mean=(0.485,), std=(0.229,)),
    ToTensorV2(),
])

num_epochs = 100

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)

        if annotation_data.ndim > 2 and annotation_data.shape[-1] != 1:
            raise ValueError('Mask has multiple channels')

        if image.shape != annotation_data.shape:
            zoom_factors = np.array(image.shape) / np.array(annotation_data.shape)
            annotation_data = zoom(annotation_data, zoom_factors, order=0)

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

# Create datasets and dataloaders for each racegender combination

# Filter racegender MB and MW
train_set_MB_MW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'MW'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_MB_MW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'MW'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Filter racegender FB and FW
train_set_FB_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['FB', 'FW'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_FB_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['FB', 'FW'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Filter racegender MW and FW
train_set_MW_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MW', 'FW'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_MW_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MW', 'FW'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Filter racegender MB and FB
train_set_MB_FB = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'FB'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_MB_FB = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'FB'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Define model architecture, optimizer, and criterion for each model
models = []
optimizers = []
criterions = []
for _ in range(4):
    model = smp.Unet(
        encoder_name="resnet18",
        encoder_weights="imagenet",
        in_channels=1,
        classes=num_classes,
    )
    model.to(device)
    models.append(model)
    optimizers.append(optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5))
    criterions.append(nn.CrossEntropyLoss())

# Create datasets and dataloaders for each racegender combination
train_sets = [train_set_MB_MW, train_set_FB_FW, train_set_MW_FW, train_set_MB_FB]
valid_sets = [valid_set_MB_MW, valid_set_FB_FW, valid_set_MW_FW, valid_set_MB_FB]

# Training loop for each model
for idx, (train_set, valid_set) in enumerate(zip(train_sets, valid_sets)):
    print(f"Training Model {idx + 1}...")
    model = models[idx]
    optimizer = optimizers[idx]
    criterion = criterions[idx]

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
        valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

        model.train()
        iou_train_list = []

        for batch_idx, (images, masks, _) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            iou_train_list.append(train_iou)

            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

        train_iou_avg = np.mean(iou_train_list)

        model.eval()
        iou_valid_list = []

        for batch_idx, (images, masks, _) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            iou_valid_list.append(valid_iou)

        valid_iou_avg = np.mean(iou_valid_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")

    print(f"Training Model {idx + 1} complete.")


Training Model 1...


/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [1/100] - Train IoU: 0.0776 - Validation IoU: 0.0490


/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [2/100] - Train IoU: 0.1066 - Validation IoU: 0.0654


...

Epoch [98/100] - Train IoU: 0.9552 - Validation IoU: 0.8866


/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [99/100] - Train IoU: 0.9433 - Validation IoU: 0.8953


/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [100/100] - Train IoU: 0.9530 - Validation IoU: 0.8946
Training Model 4 complete.


In [ ]:
# Create test datasets for each racegender combination
test_set_MB_MW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'MW'])], test_pairs,
    num_classes, transforms=test_augmentations
)
test_set_FB_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['FB', 'FW'])], test_pairs,
    num_classes, transforms=test_augmentations
)
test_set_MW_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MW', 'FW'])], test_pairs,
    num_classes, transforms=test_augmentations
)
test_set_MB_FB = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'FB'])], test_pairs,
    num_classes, transforms=test_augmentations
)

# Calculate IoU for each group using test data
test_sets = [test_set_MB_MW, test_set_FB_FW, test_set_MW_FW, test_set_MB_FB]
for idx, test_set in enumerate(test_sets):
    print(f"Calculating IoU for Test Set {idx + 1}...")
    model = models[idx]
    test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

    model.eval()
    iou_test_list = []

    for batch_idx, (images, masks, _) in enumerate(test_loader):
        images, masks = images.to(device), masks.to(device)
        with torch.no_grad():
            outputs = model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        test_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )
        iou_test_list.append(test_iou)

    test_iou_avg = np.mean(iou_test_list)
    print(f"Test Set {idx + 1} IoU: {test_iou_avg:.4f}")


Calculating IoU for Test Set 1...


/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Test Set 1 IoU: 0.6634
Calculating IoU for Test Set 2...


/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Test Set 2 IoU: 0.7119
Calculating IoU for Test Set 3...


/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Test Set 3 IoU: 0.6372
Calculating IoU for Test Set 4...


/tmp/ipykernel_218640/459124180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Test Set 4 IoU: 0.6388


In [ ]:

save_dir = "/home/ealam/MDCL"

# Save models with specified format
for idx, (model, test_set) in enumerate(zip(models, test_sets)):
    racegenders_involved = ["MB_MW", "FB_FW", "MW_FW", "MB_FB"][idx]
    model_type = "Hip"  # Change this to "Knee" if applicable
    model_name = f"{racegenders_involved}_{model_type}_baseline.pth"
    save_path = os.path.join(save_dir, model_name)
    torch.save({
        'racegenders_involved': racegenders_involved,
        'model_type': model_type,
        'state_dict': model.state_dict()
    }, save_path)

    print(f"Model saved: {save_path}")


Model saved: /home/ealam/MDCL/MB_MW_Hip_baseline.pth
Model saved: /home/ealam/MDCL/FB_FW_Hip_baseline.pth
Model saved: /home/ealam/MDCL/MW_FW_Hip_baseline.pth
Model saved: /home/ealam/MDCL/MB_FB_Hip_baseline.pth


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from scipy.ndimage import zoom
from operator import itemgetter
from sklearn.metrics import jaccard_score

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

num_classes = 7
batch_size = 8
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"
test_augmentations = A.Compose([
    A.Resize(height=256, width=256),
    A.Normalize(mean=(0.485,), std=(0.229,)),
    ToTensorV2(),
])

num_epochs = 100

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)

        if annotation_data.ndim > 2 and annotation_data.shape[-1] != 1:
            raise ValueError('Mask has multiple channels')

        if image.shape != annotation_data.shape:
            zoom_factors = np.array(image.shape) / np.array(annotation_data.shape)
            annotation_data = zoom(annotation_data, zoom_factors, order=0)

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

# Create datasets and dataloaders for each racegender combination

# Filter racegender MB and MW
train_set_MB_MW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'MW'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_MB_MW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'MW'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Filter racegender FB and FW
train_set_FB_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['FB', 'FW'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_FB_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['FB', 'FW'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Filter racegender MW and FW
train_set_MW_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MW', 'FW'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_MW_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MW', 'FW'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Filter racegender MB and FB
train_set_MB_FB = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'FB'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_MB_FB = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'FB'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Define model architecture, optimizer, and criterion for each model
models = []
optimizers = []
criterions = []
for _ in range(4):
    model = smp.Unet(
        encoder_name="resnet18",
        encoder_weights="imagenet",
        in_channels=1,
        classes=num_classes,
    )
    model.to(device)
    models.append(model)
    optimizers.append(optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5))
    criterions.append(nn.CrossEntropyLoss())

# Create datasets and dataloaders for each racegender combination
train_sets = [train_set_MB_MW, train_set_FB_FW, train_set_MW_FW, train_set_MB_FB]
valid_sets = [valid_set_MB_MW, valid_set_FB_FW, valid_set_MW_FW, valid_set_MB_FB]

# Training loop for each model
for idx, (train_set, valid_set) in enumerate(zip(train_sets, valid_sets)):
    print(f"Training Model {idx + 1}...")
    model = models[idx]
    optimizer = optimizers[idx]
    criterion = criterions[idx]

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
        valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

        model.train()
        iou_train_list = []

        for batch_idx, (images, masks, _) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            iou_train_list.append(train_iou)

            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

        train_iou_avg = np.mean(iou_train_list)

        model.eval()
        iou_valid_list = []

        for batch_idx, (images, masks, _) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            iou_valid_list.append(valid_iou)

        valid_iou_avg = np.mean(iou_valid_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")

    print(f"Training Model {idx + 1} complete.")

# Define the teacher model architecture
teacher_model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    in_channels=1,
    classes=num_classes,
)
teacher_model.to(device)

# Combine all train sets
combined_train_set = torch.utils.data.ConcatDataset([train_set_MB_MW, train_set_FB_FW, train_set_MW_FW, train_set_MB_FB])

# Training loop for the teacher model
print("Training Teacher Model...")

for epoch in range(num_epochs):
    train_loader = DataLoader(combined_train_set, batch_size=batch_size, shuffle=True, num_workers=2)

    teacher_model.train()
    iou_train_list = []

    for batch_idx, (images, masks, _) in enumerate(train_loader):
        if images is None:
            continue

        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = teacher_model(images)
        predicted_masks = torch.argmax(outputs, dim=1)

        train_iou = jaccard_score(
            masks.argmax(dim=1).cpu().numpy().flatten(),
            predicted_masks.cpu().numpy().flatten(),
            average='micro'
        )
        iou_train_list.append(train_iou)

        loss = criterion(outputs, masks.argmax(dim=1))
        loss.backward()
        optimizer.step()

    train_iou_avg = np.mean(iou_train_list)

    print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f}")

print("Training Teacher Model complete.")


Training Model 1...


/tmp/ipykernel_218640/1151518180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/1151518180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/1151518180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [1/100] - Train IoU: 0.0775 - Validation IoU: 0.0489


/tmp/ipykernel_218640/1151518180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/1151518180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/1151518180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [2/100] - Train IoU: 0.1067 - Validation IoU: 0.0645


...

Epoch [98/100] - Train IoU: 0.1276


/tmp/ipykernel_218640/1151518180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/1151518180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [99/100] - Train IoU: 0.1273


/tmp/ipykernel_218640/1151518180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/1151518180.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [100/100] - Train IoU: 0.1286
Training Teacher Model complete.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from scipy.ndimage import zoom
from sklearn.metrics import jaccard_score

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

num_classes = 7
batch_size = 8
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"
test_augmentations = A.Compose([
    A.Resize(height=256, width=256),
    A.Normalize(mean=(0.485,), std=(0.229,)),
    ToTensorV2(),
])

num_epochs = 100

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)

        if annotation_data.ndim > 2 and annotation_data.shape[-1] != 1:
            raise ValueError('Mask has multiple channels')

        if image.shape != annotation_data.shape:
            zoom_factors = np.array(image.shape) / np.array(annotation_data.shape)
            annotation_data = zoom(annotation_data, zoom_factors, order=0)

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

# Define model architecture, optimizer, and criterion for the teacher model
teacher_model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    in_channels=1,
    classes=num_classes,
)
teacher_model.to(device)
teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=0.0001, weight_decay=1e-5)
teacher_criterion = nn.CrossEntropyLoss()

# Create datasets and dataloaders for each racegender combination
racegenders = ['MB', 'MW', 'FB', 'FW']
train_loaders = {}
valid_loaders = {}

for racegender in racegenders:
    train_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df[metadata_df['racegender'] == racegender], train_pairs,
        num_classes, transforms=test_augmentations
    )
    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df[metadata_df['racegender'] == racegender], valid_pairs,
        num_classes, transforms=test_augmentations
    )
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)
    train_loaders[racegender] = train_loader
    valid_loaders[racegender] = valid_loader

# Training loop for the teacher model
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")

    # Shuffle the racegenders list for each epoch
    random.shuffle(racegenders)

    # Initialize a list to store train IoU for each racegender category
    train_iou_per_racegender = []

    for racegender in racegenders:
        print(f"Training racegender: {racegender}")
        train_loader = train_loaders[racegender]

        teacher_model.train()
        iou_train_list = []

        for batch_idx, (images, masks, _) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            teacher_optimizer.zero_grad()
            outputs = teacher_model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            iou_train_list.append(train_iou)

            loss = teacher_criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            teacher_optimizer.step()

        train_iou_avg = np.mean(iou_train_list)
        train_iou_per_racegender.append(train_iou_avg)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Racegender: {racegender} - Train IoU: {train_iou_avg:.4f}")

    # Calculate the average train IoU across all racegender categories for this epoch
    average_train_iou = np.mean(train_iou_per_racegender)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Train IoU: {average_train_iou:.4f}")


Epoch [1/100]
Training racegender: MW


/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [1/100] - Racegender: MW - Train IoU: 0.0779
Training racegender: FB


/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [1/100] - Racegender: FB - Train IoU: 0.1253
Training racegender: FW


...

Epoch [100/100] - Racegender: FB - Train IoU: 0.9708
Training racegender: MB


/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [100/100] - Racegender: MB - Train IoU: 0.9748
Training racegender: MW


/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [100/100] - Racegender: MW - Train IoU: 0.9781
Epoch [100/100] - Average Train IoU: 0.9743


In [ ]:
evaluation_results = []


for racegender in racegenders:
    train_loader = train_loaders[racegender]
    racegender_iou = evaluate_model(teacher_model, train_loader, device)
    evaluation_results.append({'Racegender': racegender, 'IOU': racegender_iou})


evaluation_results_df = pd.DataFrame(evaluation_results)


evaluation_results_sorted = evaluation_results_df.sort_values(by='IOU', ascending=False)

print("Evaluation results sorted by IOU:")
print(evaluation_results_sorted)


/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/3451063764.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returni

Evaluation results sorted by IOU:
  Racegender       IOU
3         MW  0.971093
2         MB  0.970778
0         FW  0.966061
1         FB  0.966061


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from scipy.ndimage import zoom
from sklearn.metrics import jaccard_score

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

num_classes = 7
batch_size = 8
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"
test_augmentations = A.Compose([
    A.Resize(height=256, width=256),
    A.Normalize(mean=(0.485,), std=(0.229,)),
    ToTensorV2(),
])

num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)

        if annotation_data.ndim > 2 and annotation_data.shape[-1] != 1:
            raise ValueError('Mask has multiple channels')

        if image.shape != annotation_data.shape:
            zoom_factors = np.array(image.shape) / np.array(annotation_data.shape)
            annotation_data = zoom(annotation_data, zoom_factors, order=0)

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

# Define model architecture, optimizer, and criterion for the teacher model
teacher_model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    in_channels=1,
    classes=num_classes,
)
teacher_model.to(device)
teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=0.0001, weight_decay=1e-5)
teacher_criterion = nn.CrossEntropyLoss()

# Create datasets and dataloaders for each racegender combination
racegenders = ['MB', 'MW', 'FB', 'FW']
train_loaders = {}
valid_loaders = {}

for racegender in racegenders:
    train_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df[metadata_df['racegender'] == racegender], train_pairs,
        num_classes, transforms=test_augmentations
    )
    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df[metadata_df['racegender'] == racegender], valid_pairs,
        num_classes, transforms=test_augmentations
    )
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)
    train_loaders[racegender] = train_loader
    valid_loaders[racegender] = valid_loader

# DataFrame to store results
results_df = pd.DataFrame(columns=['Epoch', 'RC1', 'RC2', 'RC3', 'RC4', 'TIOU1', 'TIOU2', 'TIOU3', 'TIOU4'])

# Training loop for the teacher model
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")

    # Shuffle the racegenders list for each epoch
    random.shuffle(racegenders)

    # Initialize a list to store train IoU for each racegender category
    train_iou_per_racegender = []

    racegender_sequence = []

    for racegender in racegenders:
        print(f"Training racegender: {racegender}")
        train_loader = train_loaders[racegender]

        teacher_model.train()
        iou_train_list = []

        for batch_idx, (images, masks, _) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            teacher_optimizer.zero_grad()
            outputs = teacher_model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            iou_train_list.append(train_iou)

            loss = teacher_criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            teacher_optimizer.step()

        train_iou_avg = np.mean(iou_train_list)
        train_iou_per_racegender.append(train_iou_avg)
        racegender_sequence.append(racegender)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Racegender: {racegender} - Train IoU: {train_iou_avg:.4f}")

    # Calculate the average train IoU across all racegender categories for this epoch
    average_train_iou = np.mean(train_iou_per_racegender)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Train IoU: {average_train_iou:.4f}")

    # Append results to DataFrame
    epoch_results = [epoch + 1] + racegender_sequence + train_iou_per_racegender
    results_df = pd.concat([results_df, pd.DataFrame([epoch_results], columns=results_df.columns)], ignore_index=True)

# Save DataFrame to CSV
results_df.to_csv('training_results.csv', index=False)
# Step 1: Load the teacher guidance metadata
teacher_guidance_metadata = pd.read_csv('training_results.csv')

# Step 2: Define the student model architecture, optimizer, and criterion
student_model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights=None,  # Initialize with random weights or pre-trained weights as needed
    in_channels=1,
    classes=num_classes,
)
student_model.to(device)
student_optimizer = optim.Adam(student_model.parameters(), lr=0.0001, weight_decay=1e-5)
student_criterion = nn.CrossEntropyLoss()

# Step 3: Create datasets and dataloaders for each racegender combination for training the student model
train_loaders_student = {}
valid_loaders_student = {}

for racegender in racegenders:
    train_set_student = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df[metadata_df['racegender'] == racegender], train_pairs,
        num_classes, transforms=test_augmentations
    )
    valid_set_student = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df[metadata_df['racegender'] == racegender], valid_pairs,
        num_classes, transforms=test_augmentations
    )
    train_loader_student = DataLoader(train_set_student, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_loader_student = DataLoader(valid_set_student, batch_size=batch_size, shuffle=False, num_workers=2)
    train_loaders_student[racegender] = train_loader_student
    valid_loaders_student[racegender] = valid_loader_student

# Step 4: Implement the training loop for the student model
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")

    # Randomly shuffle the racegenders list for each epoch
    random.shuffle(racegenders)

    # Initialize a list to store train IoU for each racegender category
    train_iou_per_racegender_student = []

    racegender_sequence_student = []

    for racegender in racegenders:
        print(f"Training racegender: {racegender}")
        train_loader_student = train_loaders_student[racegender]

        student_model.train()
        iou_train_list_student = []

        for batch_idx, (images, masks, _) in enumerate(train_loader_student):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            student_optimizer.zero_grad()
            outputs_student = student_model(images)
            predicted_masks_student = torch.argmax(outputs_student, dim=1)

            train_iou_student = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks_student.cpu().numpy().flatten(),
                average='micro'
            )
            iou_train_list_student.append(train_iou_student)

            loss_student = student_criterion(outputs_student, masks.argmax(dim=1))
            loss_student.backward()
            student_optimizer.step()

        train_iou_avg_student = np.mean(iou_train_list_student)
        train_iou_per_racegender_student.append(train_iou_avg_student)
        racegender_sequence_student.append(racegender)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Racegender: {racegender} - Train IoU: {train_iou_avg_student:.4f}")

    # Calculate the average train IoU across all racegender categories for this epoch
    average_train_iou_student = np.mean(train_iou_per_racegender_student)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Train IoU: {average_train_iou_student:.4f}")

    # Compare student's average IoU with teacher's average IoU from corresponding epoch
    teacher_iou_epoch_avg = teacher_guidance_metadata.loc[teacher_guidance_metadata['Epoch'] == epoch + 1, 'TIOU1':'TIOU4'].mean().values.mean()
    if average_train_iou_student > teacher_iou_epoch_avg:
        print("Student's IoU is better than Teacher's IoU. Proceeding to next epoch.")
    else:
        print("Student's IoU is not better than Teacher's IoU. Using teacher guidance for next epoch.")

        # Using teacher guidance racegender info for next epoch
        racegenders = list(teacher_guidance_metadata.loc[teacher_guidance_metadata['Epoch'] == epoch + 1, 'RC1':'RC4'].values[0])

# Save DataFrame to CSV
results_df.to_csv('student_training_results.csv', index=False)


Epoch [1/10]
Training racegender: MW


/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [1/10] - Racegender: MW - Train IoU: 0.0779
Training racegender: FB


/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [1/10] - Racegender: FB - Train IoU: 0.1247
Training racegender: FW


...

Epoch [10/10] - Racegender: MB - Train IoU: 0.7141
Training racegender: FW


/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [10/10] - Racegender: FW - Train IoU: 0.7238
Training racegender: FB


/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [10/10] - Racegender: FB - Train IoU: 0.7307
Epoch [10/10] - Average Train IoU: 0.7162
Student's IoU is not better than Teacher's IoU. Using teacher guidance for next epoch.


In [ ]:
# Step 1: Load the teacher guidance metadata
teacher_guidance_metadata = pd.read_csv('training_results.csv')

# Step 2: Define the student model architecture, optimizer, and criterion
student_model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights=None,  # Initialize with random weights or pre-trained weights as needed
    in_channels=1,
    classes=num_classes,
)
student_model.to(device)
student_optimizer = optim.Adam(student_model.parameters(), lr=0.0001, weight_decay=1e-5)
student_criterion = nn.CrossEntropyLoss()

# Step 3: Create datasets and dataloaders for each racegender combination for training the student model
train_loaders_student = {}
valid_loaders_student = {}

for racegender in racegenders:
    train_set_student = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df[metadata_df['racegender'] == racegender], train_pairs,
        num_classes, transforms=test_augmentations
    )
    valid_set_student = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df[metadata_df['racegender'] == racegender], valid_pairs,
        num_classes, transforms=test_augmentations
    )
    train_loader_student = DataLoader(train_set_student, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_loader_student = DataLoader(valid_set_student, batch_size=batch_size, shuffle=False, num_workers=2)
    train_loaders_student[racegender] = train_loader_student
    valid_loaders_student[racegender] = valid_loader_student

# Step 4: Implement the training loop for the student model
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")

    # Reverse the racegenders list to follow teacher guidance in reverse order
    racegenders_reverse = racegenders[::-1]

    # Initialize a list to store train IoU for each racegender category
    train_iou_per_racegender_student = []

    racegender_sequence_student = []

    for idx, racegender in enumerate(racegenders_reverse):
        print(f"Training racegender: {racegender}")
        train_loader_student = train_loaders_student[racegender]

        student_model.train()
        iou_train_list_student = []

        for batch_idx, (images, masks, _) in enumerate(train_loader_student):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            student_optimizer.zero_grad()
            outputs_student = student_model(images)
            predicted_masks_student = torch.argmax(outputs_student, dim=1)

            train_iou_student = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks_student.cpu().numpy().flatten(),
                average='micro'
            )
            iou_train_list_student.append(train_iou_student)

            loss_student = student_criterion(outputs_student, masks.argmax(dim=1))
            loss_student.backward()
            student_optimizer.step()

        train_iou_avg_student = np.mean(iou_train_list_student)
        train_iou_per_racegender_student.append(train_iou_avg_student)
        racegender_sequence_student.append(racegender)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Racegender: {racegender} - Train IoU: {train_iou_avg_student:.4f}")

    # Calculate the average train IoU across all racegender categories for this epoch
    average_train_iou_student = np.mean(train_iou_per_racegender_student)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Train IoU: {average_train_iou_student:.4f}")

    # Get teacher guidance racegender info for the current epoch
    teacher_racegenders = teacher_guidance_metadata.loc[teacher_guidance_metadata['Epoch'] == num_epochs - epoch, 'RC1':'RC4'].values[0]

    # Set the racegenders list to teacher guidance racegender info for the next epoch
    racegenders = list(teacher_racegenders)

# Save DataFrame to CSV
results_df.to_csv('student_training_results.csv', index=False)


Epoch [1/10]
Training racegender: FB


/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [1/10] - Racegender: FB - Train IoU: 0.1581
Training racegender: FW


/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [1/10] - Racegender: FW - Train IoU: 0.1859
Training racegender: MW


...

Epoch [10/10] - Racegender: MW - Train IoU: 0.8126
Training racegender: FW


/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [10/10] - Racegender: FW - Train IoU: 0.8255
Training racegender: MB


/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/1256867411.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [10/10] - Racegender: MB - Train IoU: 0.8772
Epoch [10/10] - Average Train IoU: 0.8470


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from scipy.ndimage import zoom
from sklearn.metrics import jaccard_score

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

num_classes = 7
batch_size = 8
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Hip/segmentation_with_racegender.csv"
test_augmentations = A.Compose([
    A.Resize(height=256, width=256),
    A.Normalize(mean=(0.485,), std=(0.229,)),
    ToTensorV2(),
])

num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)

        if annotation_data.ndim > 2 and annotation_data.shape[-1] != 1:
            raise ValueError('Mask has multiple channels')

        if image.shape != annotation_data.shape:
            zoom_factors = np.array(image.shape) / np.array(annotation_data.shape)
            annotation_data = zoom(annotation_data, zoom_factors, order=0)

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

# Define teacher model architecture, optimizer, and criterion
teacher_model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    in_channels=1,
    classes=num_classes,
)
teacher_model.to(device)
teacher_optimizer = optim.Adam(teacher_model.parameters(), lr=0.0001, weight_decay=1e-5)
teacher_criterion = nn.CrossEntropyLoss()

# Define student model architecture, optimizer, and criterion
student_model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights=None,  # Student model starts from scratch
    in_channels=1,
    classes=num_classes,
)
student_model.to(device)
student_optimizer = optim.Adam(student_model.parameters(), lr=0.0001, weight_decay=1e-5)
student_criterion = nn.CrossEntropyLoss()

# Create datasets and dataloaders for each racegender combination
racegenders = ['MB', 'MW', 'FB', 'FW']
train_loaders = {}
valid_loaders = {}

for racegender in racegenders:
    train_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df[metadata_df['racegender'] == racegender], train_pairs,
        num_classes, transforms=test_augmentations
    )
    valid_set = MulticlassHipSegmentationDataset(
        img_root, mask_root, metadata_df[metadata_df['racegender'] == racegender], valid_pairs,
        num_classes, transforms=test_augmentations
    )
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)
    train_loaders[racegender] = train_loader
    valid_loaders[racegender] = valid_loader

# DataFrame to store results
results_df = pd.DataFrame(columns=['Epoch', 'RC1', 'RC2', 'RC3', 'RC4', 'TIOU1', 'TIOU2', 'TIOU3', 'TIOU4'])

# Training loop for the teacher-student curriculum learning
for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")

    # Shuffle the racegenders list for each epoch
    random.shuffle(racegenders)

    # Initialize a list to store train IoU for each racegender category
    train_iou_per_racegender = []

    racegender_sequence = []

    for racegender in racegenders:
        print(f"Training racegender: {racegender}")
        train_loader = train_loaders[racegender]

        # Switch between teacher and student models for curriculum learning
        model = teacher_model if epoch < num_epochs // 2 else student_model
        optimizer = teacher_optimizer if epoch < num_epochs // 2 else student_optimizer
        criterion = teacher_criterion if epoch < num_epochs // 2 else student_criterion

        model.train()
        iou_train_list = []

        for batch_idx, (images, masks, _) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            iou_train_list.append(train_iou)

            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

        train_iou_avg = np.mean(iou_train_list)
        train_iou_per_racegender.append(train_iou_avg)
        racegender_sequence.append(racegender)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Racegender: {racegender} - Train IoU: {train_iou_avg:.4f}")

    # Calculate the average train IoU across all racegender categories for this epoch
    average_train_iou = np.mean(train_iou_per_racegender)
    print(f"Epoch [{epoch + 1}/{num_epochs}] - Average Train IoU: {average_train_iou:.4f}")

    # Sort training samples based on IoU scores (if epoch is within the first half)
    if epoch < num_epochs // 2:
        sorted_indices = np.argsort(iou_train_list)[::-1]  # Sort in descending order
        train_pairs = [train_pairs[i] for i in sorted_indices]

    # Append results to DataFrame
    epoch_results = [epoch + 1] + racegender_sequence + train_iou_per_racegender
    results_df = pd.concat([results_df, pd.DataFrame([epoch_results], columns=results_df.columns)], ignore_index=True)

    # Save the trained student model after every epoch
    torch.save(student_model.state_dict(), f'student_model_epoch_{epoch + 1}.pt')

# Save DataFrame to CSV
results_df.to_csv('training_results.csv', index=False)


Epoch [1/10]
Training racegender: MW


/tmp/ipykernel_218640/3927485296.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/3927485296.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [1/10] - Racegender: MW - Train IoU: 0.0807
Training racegender: FB


/tmp/ipykernel_218640/3927485296.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [1/10] - Racegender: FB - Train IoU: 0.1045
Training racegender: FW


...

Epoch [10/10] - Racegender: MW - Train IoU: 0.4804
Training racegender: FW


/tmp/ipykernel_218640/3927485296.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/3927485296.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [10/10] - Racegender: FW - Train IoU: 0.5135
Training racegender: FB


/tmp/ipykernel_218640/3927485296.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_218640/3927485296.py:86: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'


Epoch [10/10] - Racegender: FB - Train IoU: 0.5204
Epoch [10/10] - Average Train IoU: 0.4879
